In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url = f"https://jateng.antaranews.com/daerah/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 6 links from page 1
Total Links: 6


In [5]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('div')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jateng',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()
    

In [7]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

6


,title,author,category,date,content,region,link
0,Pemkab Batang larang ASN bagikan komentar cap...,Kutnadi\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 10:24 WIB\nRabu, 27 Se...",Content not found,jateng,https://jateng.antaranews.com/berita/509223/pe...
1,Bank Jateng Cilacap kucurkan Rp969 juta untuk ...,ksm\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 09:58 WIB\nRabu, 27 Se...",Content not found,jateng,https://jateng.antaranews.com/berita/509217/ba...
2,Empat tersangka pengiriman rokok ilegal dilim...,Immanuel Citra Senjaya\n\n\t\t\t\t\t\t\t\t ...,Category not found,"Rabu, 27 September 2023 09:42 WIB\nRabu, 27 Se...",Content not found,jateng,https://jateng.antaranews.com/berita/509211/em...
3,Bandara Ahmad Yani resmi layani rute Semarang...,Wisnu Adhi Nugroho\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 08:44 WIB\nRabu, 27 Se...",Content not found,jateng,https://jateng.antaranews.com/berita/509202/ba...
4,Pemkab Batang terima kuota 1.587 formasi PPPK...,Kutnadi\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 09:43 WIB\nRabu, 27 Se...",Content not found,jateng,https://jateng.antaranews.com/berita/509214/pe...


In [8]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/jateng_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')